In [1]:
import copy
import sys
import os
from functools import partial

import geopandas as gpd
from pandas import IndexSlice as idx
import numpy as np
import tensorflow as tf
import gpflow
from tensorflow.python.data.ops.iterator_ops import OwnedIterator as DatasetOwnedIterator

#from metrics import fixed_top_X
#from model_runner import run_adam

code_dir = '/cluster/home/kheuto01/code/zero-inflated-gp/'
sys.path.append(code_dir)
code_dir = '/cluster/home/kheuto01/code/opioid-overdose-models/perturbations/'
sys.path.append(code_dir)
code_dir = '/cluster/home/kheuto01/code/opioid-overdose-models/diff_bpr'
sys.path.append(code_dir)

from perturbations import perturbed
from bpr import bpr_variable_k_no_ties

from onoffgpf import OnOffSVGP, OnOffSVGPPoiMC, OnOffLikelihood,OnOffSVGPBatch
gpflow.config.default_float()

2023-05-23 12:34:00.499122: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-05-23 12:34:00.501202: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-05-23 12:34:00.551031: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-05-23 12:34:00.552156: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-23 12:34:03.949107: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Co

numpy.float64

In [2]:
data_dir='/cluster/tufts/hugheslab/datasets/NSF_OD/results_20220606_update/'

In [3]:
data_path = os.path.join(data_dir, 'clean_annual_tract')
timestep_col = 'timestep'
geography_col = 'geoid'
outcome_col = 'deaths'
last_train_year = 2018
first_train_year = 2000
test_years = 2
use_auto = False
use_svi = True
seed=360
inducing_points = 200
learning_rate = 0.001
minibatch_size = 100

sigma = 0.05
bpr_samples = 37
noise='normal'

In [4]:
x_idx_cols = [geography_col, 'lat','lon', timestep_col,
              'theme_1_pc', 'theme_2_pc', 'theme_3_pc', 'theme_4_pc',
              'svi_pctile','year',
              'neighbor_t', 'self_t-1']
y_idx_cols = [geography_col, timestep_col, outcome_col]
features_only = ['lat','lon', timestep_col,
                 'theme_1_pc', 'theme_2_pc', 'theme_3_pc', 'theme_4_pc',
                 'svi_pctile',
                 'neighbor_t', 'self_t-1']

data_gdf = gpd.read_file(data_path)



In [5]:
time_window = 10
first_train_eval_year = 2014
last_train_eval_year = 2018

In [6]:
multiindexed_gdf = data_gdf.set_index(['geoid','year'])
num_geoids = len(data_gdf['geoid'].unique())

train_shape = (num_geoids, time_window, len(['deaths']))

xs =[]
ys = []

for eval_year in range(first_train_eval_year, last_train_eval_year+1):
    
    train_x_df = multiindexed_gdf.loc[idx[:,eval_year-time_window:eval_year-1], 'deaths']
    train_y_df = multiindexed_gdf.loc[idx[:,eval_year], 'deaths']
    
    train_x_vals = train_x_df.values.reshape(train_shape)

    train_y_vals = train_y_df.values
    
    xs.append(train_x_vals)
    ys.append(train_y_vals)
    
    
x_BSTD = np.stack(xs,axis=0)
y_BS = np.stack(ys)

x_BSTD = tf.convert_to_tensor(x_BSTD, dtype=tf.float32)
y_BS = tf.convert_to_tensor(y_BS, dtype=tf.float32)

B, S, T, D = x_BSTD.shape

assert(B==len( range(first_train_eval_year, last_train_eval_year+1)))
assert(S==num_geoids)
assert(T==time_window)
assert(D==1)

# Reshape the training data to flatten the dimensions
x_BSF_flat = tf.reshape(x_BSTD, (B, S, T * D), )

2023-05-23 12:34:41.084251: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:266] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2023-05-23 12:34:41.084291: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:168] retrieving CUDA diagnostic information for host: p1cmp078.pax.tufts.edu
2023-05-23 12:34:41.084299: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:175] hostname: p1cmp078.pax.tufts.edu
2023-05-23 12:34:41.084414: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:199] libcuda reported version is: 515.65.1
2023-05-23 12:34:41.084445: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:203] kernel reported version is: 515.65.1
2023-05-23 12:34:41.084451: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:309] kernel version seems to match DSO: 515.65.1


In [7]:
test_year=2019
test_x_df = multiindexed_gdf.loc[idx[:,test_year-time_window:test_year-1], 'deaths']
test_y_df = multiindexed_gdf.loc[idx[:,test_year], 'deaths']

test_x_vals = test_x_df.values.reshape(train_shape)

test_y_vals = test_y_df.values

test_x_BSTD = np.expand_dims(test_x_vals,axis=0)
test_y_BS = np.expand_dims(test_y_vals,axis=0)

test_x_BSTD = tf.convert_to_tensor(test_x_BSTD, dtype=tf.float32)
test_y_BS = tf.convert_to_tensor(test_y_BS, dtype=tf.float32)


# Reshape the training data to flatten the dimensions
test_x_BSF_flat = tf.reshape(test_x_BSTD, (1, S, T * D), )

In [46]:
def top_k_idx(input_BD, **kwargs):
    
    _, idx_BD = tf.math.top_k(input_BD, **kwargs)
    input_depth = input_BD.shape[-1]
    one_hot_idx_BKD = tf.one_hot(idx_BD, input_depth)
    #Sum over k dimension so we dont have to worry about sorting
    k_hot_idx_BD = tf.reduce_sum(one_hot_idx_BKD, axis=-2)
    
    
    return k_hot_idx_BD

top_100_idx = partial(top_k_idx, k=100)
perturbed_top_100 = perturbed(top_100_idx,
                         num_samples=bpr_samples,
                         sigma=sigma,
                         noise=noise,
                         batched=True)

class PerturbedBPRModel(tf.keras.Model):
    
    def __init__(self, perturbed_top_k_func, k=100):
        """k should match the k baked into the perturbed top_k func. 
        we need k for when performing exact top k in evaluation step."""
        super(PerturbedBPRModel, self).__init__()
        self.perturbed_top_k_func = perturbed_top_k_func
        self.k = k
        self.hidden = tf.keras.layers.Dense(10, activation='relu')
        self.output_layer =  tf.keras.layers.Dense(1, activation=None)
        
    def call(self, inputs):
        
        intermediate = self.hidden(inputs)
        outputs = self.output_layer(intermediate)
        # squeeze away feature dimension
        outputs = tf.squeeze(outputs, axis=-1)
        return outputs

    def train_step(self, data):
        # Unpack the data. Its structure depends on your model and
        # on what you pass to `fit()`.
        x, y = data

        with tf.GradientTape() as tape:
            y_pred = self(x, training=True)  # Forward pass
            top_100_indicators = self.perturbed_top_k_func(y_pred)
            true_top_100_val, true_top_100_idx = tf.math.top_k(y, k=self.k)
            
            denominator = tf.reduce_sum(true_top_100_val, axis=-1)
            numerator = tf.reduce_sum(top_100_indicators*y, axis=-1)
            
            # Compute the loss value
            # (the loss function is configured in `compile()`)
            loss = self.compiled_loss(numerator, denominator, regularization_losses=self.losses)

        # Compute gradients
        trainable_vars = self.trainable_variables
        gradients = tape.gradient(loss, trainable_vars)
        # Update weights
        self.optimizer.apply_gradients(zip(gradients, trainable_vars))
        # Update metrics (includes the metric that tracks the loss)
        self.compiled_metrics.update_state(y, y_pred)
        # Return a dict mapping metric names to current value
        return {m.name: m.result() for m in self.metrics}
    
    def test_step(self, data):
        # Unpack the data
        x, y = data
        # Compute predictions
        y_pred = self(x, training=False)  # Forward pass
        # use discrete topk to simulate making a decision
        _, pred_100_idx = tf.math.top_k(y_pred, k=self.k)
        true_top_100_val, true_top_100_idx = tf.math.top_k(y, k=self.k)

        denominator = tf.reduce_sum(true_top_100_val, axis=-1)
        numerator = tf.reduce_sum(tf.gather(y, pred_100_idx, batch_dims=-1), axis=-1)

        # Compute the loss value
        # (the loss function is configured in `compile()`)
        self.compiled_loss(numerator, denominator, regularization_losses=self.losses)

        # Update the metrics.
        self.compiled_metrics.update_state(y, y_pred)
        return {m.name: m.result() for m in self.metrics}

In [47]:
model = PerturbedBPRModel(perturbed_top_100)

In [48]:
optimizer = tf.keras.optimizers.Adam(learning_rate=0.005)

# Compile the model
def weird_loss(a,b):
    return -a/b

model.compile(optimizer=optimizer, loss=weird_loss)

In [52]:
# Train the model
model.fit(x_BSF_flat, y_BS, epochs=100, batch_size=5, validation_data=(test_x_BSF_flat, test_y_BS))  

Epoch 1/100
1/1 [==============================] - 0s 92ms/step - loss: -0.4319 - val_loss: -0.4551
Epoch 2/100
1/1 [==============================] - 0s 64ms/step - loss: -0.4320 - val_loss: -0.4551
Epoch 3/100
1/1 [==============================] - 0s 67ms/step - loss: -0.4330 - val_loss: -0.4615
Epoch 4/100
1/1 [==============================] - 0s 63ms/step - loss: -0.4332 - val_loss: -0.4530
Epoch 5/100
1/1 [==============================] - 0s 64ms/step - loss: -0.4330 - val_loss: -0.4551
Epoch 6/100
1/1 [==============================] - 0s 62ms/step - loss: -0.4334 - val_loss: -0.4551
Epoch 7/100
1/1 [==============================] - 0s 62ms/step - loss: -0.4336 - val_loss: -0.4551
Epoch 8/100
1/1 [==============================] - 0s 63ms/step - loss: -0.4345 - val_loss: -0.4573
Epoch 9/100
1/1 [==============================] - 0s 65ms/step - loss: -0.4324 - val_loss: -0.4594
Epoch 10/100
1/1 [==============================] - 0s 65ms/step - loss: -0.4315 - val_loss: -0.4466

1/1 [==============================] - 0s 61ms/step - loss: -0.4232 - val_loss: -0.4551
Epoch 83/100
1/1 [==============================] - 0s 62ms/step - loss: -0.4215 - val_loss: -0.4551
Epoch 84/100
1/1 [==============================] - 0s 61ms/step - loss: -0.4192 - val_loss: -0.4551
Epoch 85/100
1/1 [==============================] - 0s 61ms/step - loss: -0.4167 - val_loss: -0.4551
Epoch 86/100
1/1 [==============================] - 0s 63ms/step - loss: -0.4208 - val_loss: -0.4551
Epoch 87/100
1/1 [==============================] - 0s 64ms/step - loss: -0.4192 - val_loss: -0.4530
Epoch 88/100
1/1 [==============================] - 0s 64ms/step - loss: -0.4196 - val_loss: -0.4530
Epoch 89/100
1/1 [==============================] - 0s 64ms/step - loss: -0.4214 - val_loss: -0.4530
Epoch 90/100
1/1 [==============================] - 0s 62ms/step - loss: -0.4227 - val_loss: -0.4637
Epoch 91/100
1/1 [==============================] - 0s 61ms/step - loss: -0.4253 - val_loss: -0.4637
Epo

In [40]:
y_pred = model(test_x_BSF_flat, training=False)  # Forward pass
# use discrete topk to simulate making a decision
_, pred_100_idx = tf.math.top_k(y_pred)
true_top_100_val, true_top_100_idx = tf.math.top_k(test_y_BS,k=100)

denominator = tf.reduce_sum(true_top_100_val, axis=-1)
numerator = tf.reduce_sum(tf.gather(test_y_BS, pred_100_idx, batch_dims=-1), axis=-1)

In [45]:
pred_100_idx

<tf.Tensor: shape=(1, 1), dtype=int32, numpy=array([[115]], dtype=int32)>

In [54]:
model.evaluate(test_x_BSF_flat, test_y_BS)

1/1 [==============================] - 0s 21ms/step - loss: -0.4936


-0.49358972907066345

In [27]:
test_y_BS.shape

TensorShape([1, 1620])